# Building an Autogen Agent with Long-Term Memory via Flotorch

This notebook demonstrates how to build an advanced Autogen agent with both short-term and long-term memory capabilities using the Flotorch platform. This allows the agent to recall information not only from the current conversation but also from previous, separate interactions.

### Prerequesit
Configure model, memory provider and API key in Flotroch console (https://console.flotorch.cloud/)

### Viewing logs
Logs can be viewed in logs tab in Flotroch console (https://console.flotorch.cloud/)

### Key Concepts:
- **Short-Term Memory**: Provided by `FlotorchAutogenSession` for session-specific context.
- **External (Long-Term) Memory**: Provided by `FlotorchAutogenMemory` for persistent knowledge across all sessions.
- **`AssistantAgent`, `RoundRobinGroupChat`**: The core Autogen components configured to leverage both memory types.

## 1. Setup and Imports

The following cells install the necessary packages, configure API credentials, and import the required components from Flotorch and Autogen, including those needed for external memory.

In [ ]:
# install flotorch autogen package
%pip install flotorch[autogen]

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
FLOTORCH_MODEL = "<flotorch model>"
USER_ID = "flotorch_user_001"
APP_ID = "flotorch_app_001"
MEMORY_PROVIDER = "<flotorch memory provider>"

In [ ]:
# Import Autogen, Flotorch and related modules
from flotorch.autogen.llm import FlotorchAutogenLLM
from flotorch.autogen.sessions import FlotorchAutogenSession
from flotorch.autogen.memory import FlotorchAutogenMemory
from autogen_core.memory import MemoryContent, MemoryMimeType

from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console



print("Imported necessary libraries successfully")


## 2. Model and Memory Configuration

We initialize the `FlotorchAutogenLLM` which will act as the brain for our agent. This is where we also set up the agent's memory systems, configuring both short-term and long-term memory to give the agent a comprehensive ability to recall information.

- **Short-Term Memory**: Uses `FlotorchAutogenSession` to remember the current conversation's flow.
- **External Memory**: Uses `FlotorchAutogenMemory` to store and retrieve key information across different sessions, acting as the agent's long-term knowledge base.

In [ ]:
model  =  FlotorchAutogenLLM(
        model_id=FLOTORCH_MODEL,
        api_key=FLOTORCH_API_KEY,
        base_url=FLOTORCH_BASE_URL,
    )

print(f"Flotorch LLM model configured")

In [ ]:
session_memory = FlotorchAutogenSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)


external_memory = FlotorchAutogenMemory(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
    name = MEMORY_PROVIDER,
    user_id = USER_ID,
    app_id  = APP_ID
)

print("Initialized session_memory and external_memory")

## 3. Agent and Team Configuration

We define the `AssistantAgent` and a `RoundRobinGroupChat` with a goal that explicitly instructs the agent to use its memory capabilities to provide accurate and context-aware responses. The agent is configured with both `model_context` (for short-term memory) and a list of `memory` providers (for long-term memory).

In [ ]:
assistant = AssistantAgent(
        name="FlotorchAssistant",
        model_client=model,
        model_context = session_memory,
        memory = [external_memory],
        system_message = """  
       You are a helpful Assistant with access to the session memory, and external memory,
       so use the session memory properly and answer the user queries.
        """ )
    
team = RoundRobinGroupChat(
        participants=[assistant],
        max_turns=1
    )
print(f"Agent and Task created successfully: {assistant.name}")

## 4. Interactive Chat

Start a chat session to interact with the agent. The agent's ability to recall information is tested, both from the current conversation (`session_memory`) and from facts it may have learned in previous runs (`external_memory`). Note how user inputs and agent responses are manually saved to the external memory after each turn.

In [ ]:
while True:
    user_input = input("user: ")

    if user_input.lower().strip() == "exit":
        break

    stream = team.run_stream(task = user_input)
    result = await Console(stream)
     # --- Add user input to memory ---
    await external_memory.add(MemoryContent(content=user_input, mime_type=MemoryMimeType.TEXT))

    # --- Add assistant response(s) to memory ---
    if hasattr(result, "messages"):
        for msg in result.messages[-1]:
            # Only save text messages (type == "TextMessage")
            if getattr(msg, "type", None) == "TextMessage":
                
                await external_memory.add(MemoryContent(content=getattr(msg, "content", ""), mime_type=MemoryMimeType.TEXT))


## Summary

This notebook detailed the implementation of a sophisticated AI agent equipped with both short-term and long-term memory.  
By leveraging **Flotorch's infrastructure**, we created an agent that not only remembers the current conversation but also retains and recalls information across separate sessions.

### Key Achievements

- **Dual-Memory Architecture** 
  Successfully integrated two distinct memory types:  
  - `FlotorchAutogenSession` for immediate, short-term conversational context.  
  - `FlotorchAutogenMemory` for persistent, long-term knowledge.

- **Enhanced Agent Intelligence** 
  The agent demonstrated its ability to recall facts from a knowledge base established in previous interactions, leading to more intelligent and informed responses.

- **Comprehensive Solution** 
  This dual-memory approach provides a powerful template for building advanced agents capable of complex, ongoing, and context-rich dialogues.